# Rekursion in anonymen Funktionen
<br>



<section>
<h4>Übersicht</h4>
<hr />
<div style="text-align: justify">Dieser Text behandelt, wie Rekursion in formalen Systemen realisiert werden kann, die keine Zuweisung kennen. Konkret wird der nicht-typisierte Lambda-Kalkül als Instanz eines solchen Systems herangezogen. Es werden Schritt für Schritt formale Werkzeuge konstruiert, die das Problem der Rekursion im nicht-typisierten Lambda-Kalkül behandeln.</div>
<br>
<br>
</section><section>
<h4>Das Problem</h4>
<hr />
<div style="text-align: justify">Wann immer wir eine rekursive Funktion definieren, muss diese Funktion Zugriff auf ihre eigene Definition haben. In einem formalen System, das Zuweisung der Form <em>Sei f =</em> e (mit <em>e</em> einem Ausdruck der Sprache des formalen Systems von <em>f</em>) erlaubt, ist das kein Problem. Im einfachen Lambda-Kalkül ist dies allerdings nicht vorgesehen; dieser kennt nur <strong>anonyme</strong> Funktionen. Dennoch gibt es eine Möglichkeit, Funktionen Zugriff auf ihre eigene Definition zu verschaffen. Diese Möglichkeit sind sog. <strong>Fixpunkt-Kombinatoren</strong>. Im Folgenden werde ich Schritt für Schritt verschiedene solcher Kombinatoren konstruieren.</div>

<br>
<br>
</section><section>
<h4>Fixpunkte: von Iteration zu Kombinatoren</h4>
<hr />
<div style="text-align: justify">Der <strong>Fixpunkt</strong> einer Funktion ist ein Wert <em>x</em>, sodass gilt <em>f(x) = x</em>. Somit gilt dann auch <em>f(x) = f(f(x)) = f(... f(f(x))...)</em>. Um zu sehen, warum das relevant für die Implementierung rekursiver Funktionen ist, schauen wir uns mal folgendes Python-Segment an, das die Fakultätsfunktion anonym implementiert:</div>

In [1]:
# get nth value of any iterable (including generators)
def iter_get_nth(iterable, n):
    for i, e in enumerate(iterable):
        if i == n: return e

# iterate function f on initial value x (default f itself)
def iterate(f, x=None):
    if x is None: x = f
    while True: x = f(x); yield x

# iterate a function f n times on initial value x (default f itself)
# return function after n iterations
def iterate_to(f, n, x=None):
    if x is None: x = f
    return iter_get_nth(iterate(f, x),n)

factorial = lambda n : iterate_to(lambda f : lambda n : 1 if not n else n * f(n-1), n)(n)

Oder etwas kompakter in Haskell:
<pre class="theme:obsidian lang:haskell decode:true">let factorial = \n -&gt; (iterate (\f -&gt; \n -&gt; if n == 0 then 1 else n * f (n-1)) undefined) !! (n+1) $ n
</pre>
<div style="text-align: justify">Wir haben hier also eine Funktion <code>iterate</code>, die eine andere Funktion <code>f</code> und einen Initialwert <code>x</code> nimmt und <code>f</code> auf dem Initialwert iteriert. Bei <em>n</em>-facher Anwendung von <code>iterate</code> erhalten wir also eine <em>n</em>-fache Anwendung von <code>f</code> auf sich selbst, wobei die erste Anwendung auf <code>x</code> stattfindet. Ferner ist das zweite Argument von <code>iterate</code> optional, wobei als Default-Wert <code>f</code> selbst definiert ist. Also gilt für <em>n ⟶ ∞ </em> mit <a href="https://en.wikipedia.org/wiki/Kleene_fixed-point_theorem">Kleenes Fixpunkt-Satz</a></div>

<p style="text-align: center;"><em><code>iterate(f) = fⁿ = f(fⁿ) = f(iterate(f))</code></em>.</p>

Also per <a href="https://de.wikipedia.org/wiki/%C3%84quivalenzrelation#Definition_einer_%C3%84quivalenzrelation">Symmetrie</a> auch
<p style="text-align: center;"><em><code>f(iterate(f)) = iterate(f)</code></em>.</p>

<div style="text-align: justify">Dies entspricht der Form eines Fixpunktes einer Funktion. Also ist <code>iterate(f)</code> der Fixpunkt von <code>f</code>.</div>
<br>

<div style="text-align: justify">An diesem Beispiel und unserem kurzen Beweis sehen wir, dass Fixpunkte genutzt werden können, um Rekursion zu implementieren. Haben wir also jetzt das Problem der Rekursion in formalen Systemen ohne Zuweisung schon gelöst? Immerhin ruft <code>factorial</code> sich ja nirgends selbst auf.
Das stimmt zwar, aber in der Python-Implementierung haben wir Zuweisung an der Stelle <code>x = f(x)</code> und die Definition der Haskell-Version von <code>iterate</code> verwendet selbst Rekursion:</div>

<pre class="theme:obsidian lang:haskell decode:true">iterate          :: (a -&gt; a) -&gt; a -&gt; [a]
iterate f x      =  x : iterate f (f x)
</pre>

<div style="text-align: justify">Wir haben die Zuweisung also nur ausgelagert und sie auf diese Weise versteckt, aber sie ist immer noch da. Können wir auch eine Funktion schreiben, die ohne Zuweisung auf sich selbst iteriert?
Wir müssten dazu eine Funktion schreiben, die ihre eigene Definition beinhaltet. Und da diese wiederum die Definition der Funktion ist, muss auch diese die Definition der Funktion beinhalten. Das können wir erreichen, indem wir die Funktion sich selbst übergeben und sie so bauen, dass sie selber sich selbst übergibt. Auf diese Weise erhält jede eingebettete Definition der Funktion den Code ihrer eigenen Definition und jeder Aufruf der Funktion gibt ihre eigene Definition weiter. Wie wäre es denn also hiermit:</div>

In [2]:
def f(g):
    return g(g)

<div style="text-align: justify">Wenn wir <code>f</code> auf <code>f</code> anwenden, rufen wir im Rumpf von <code>f</code> <code>f</code> mit <code>f</code> auf, was wiederum <code>f</code> mit <code>f</code> aufruft etc. Probieren wir das mal in Python aus:</div>

In [3]:
f(f)

RecursionError: maximum recursion depth exceeded

<div style="text-align: justify">Naja, hatten wir wohl erwartet. Aber im Prinzip funktioniert unser Ansatz, nur bekommen wir ein Problem mit Pythons <strong>Auswertungsstrategie</strong>. Diese ist nämlich <strong>streng</strong> (genauer: <strong>applikativ</strong>). Was ja bedeutet, dass Argumente immer zuerst ausgewertet werden, bevor sie übergeben werden. Das führt hier zu einer verfrühten Endlosrekursion. Rein formal wollen wir genau das aber ja erreichen.
Wenn wir eine Funktion mit Fallunterscheidung wie die Fakultätsfunktion schreiben, verschwindet das Problem, da nur True-Branches in Python evaluiert werden:</div>

In [5]:
def factorial_(f, n):
    return 1 if not n else n*f(f, n-1)

factorial_(factorial_, 5)

120

<div style="text-align: justify">Hier haben wir das ursprüngliche Problem der Rekursion in anonymen Funktionen eigentlich schon gelöst. Und um zu zeigen, dass diese Lösung auch wirklich mit solchen funktioniert:</div>

In [6]:
(lambda f : lambda n : 1 if not n else n*f(f)(n-1))((lambda f : lambda n : 1 if not n else n*f(f)(n-1)))(5)

120

<div style="text-align: justify">Die Funktion <code>factorial_</code> bzw. <code>lambda f : lambda n : 1 if not n else n*f(f)(n-1)</code> iteriert also nun auf sich selber, wenn sie sich selbst übergeben wird. Das Ganze ist aber noch ein wenig unelegant. Der Aufruf <code>f(f)(n-1)</code> ist unschön und die Tatsache, dass wir keine Funktion wie <code>iterate</code> mehr haben ebenso.
Das zweite Problem können wir ganz einfach lösen, indem wir eine Funktion schreiben, die eine Argument-Funktion sich selbst übergibt. Diese Funktion heißt im Lambda-Kalkül <strong>ω-Kombinator:</strong></div>

In [7]:
from functools import partial
omega = lambda x : partial(x, x) # partial, da Python nativ kein Currying macht
omega(factorial_)(5)

120

<div style="text-align: justify">Wie können wir aber das erste Problem beheben? Wir wollen, dass wir nicht <code>f(f)(x)</code>  schreiben müssen, sondern einfach <code>A(x)</code> . Dafür muss <code>A</code> ein Ausdruck sein, der selbstständig die Definition von <code>f</code> an <code>f</code> weitergibt und <code>x</code> verarbeitet. Wenn wir die Fakultätsfunktion wie folgt noch einmal definieren -- diesmal mit Currying -- muss also gelten, dass, wenn an der Stelle <code>f(f)(n-1)</code> der Ausdruck <code>A(n-1)</code> stünde, es ungefähr zu <code>inner(n-1)</code> evaluiert, denn <code>inner</code> verarbeitet ja <code>x</code> (hier <code>n-1</code>).</div>

In [8]:
def factorial(f):
    def inner(n):
        return 1 if not n else n*f(f)(n-1)
    return inner

<div style="text-align: justify">Aber derzeit tut es das natürlich nicht, denn <code>n-1</code> (bzw. <code>x</code>) ist eine Zahl und und für den Parameter <code>f</code> wird eine Funktion erwartet (nämlich <code>factorial_</code>).
<br>
Wir brauchen also einen Ausdruck <code>A</code> an der Stelle <code>f(f)</code>, der die Ableitung <em><code>A x ⟶ inner x ⟶ A x' ⟶ inner x' ⟶ A x'' ⟶ inner x'' ...</code></em> erfüllt. Wenn wir den Ausdruck <code>A</code> innerhalb von <code>inner</code> wieder benutzen wollen, dann muss in <code>A</code> ja irgendwie <code>factorial_</code> vorkommen, denn über <code>factorial_</code> injizieren wir die Funktion <code>f</code> in <code>inner</code> hinein. Und <code>A</code> soll ja da stehen, wo jetzt <code>f(f)</code> steht. Also müssen wir den Ausdruck, der stets die Definition von <code>factorial_</code> weitergibt, an <code>factorial_</code> übergeben. Somit wissen wir schonmal, dass <code>A</code> die Form <code>factorial_ B</code> hat. <code>B</code> wird im nächsten Schritt dort stehen, wo jetzt <code>A</code> steht: <em><code>factorial_ B ⟶ factorial_[f/B]</code></em> (<em>E[x/y]</em> bedeutet, "substituiere <em>x</em> in <em>E</em> durch <em>y</em>"). <code>B</code> muss sich jetzt aber verhalten wie <code>A</code>, da es jetzt an <code>A</code>’s Position steht. Also ist <code>B</code> ein Teilausdruck von <code>A</code> und trotzdem identisch zu <code>A</code>. Demnach gilt</div>
<br>
\begin{aligned}
\text{A} &= \text{factorial_ B} &&\text{Definition von A}\\
         &= \text{B}                     &&\text{Transitivität (B = A = factorial_ B)}\\
\\
\implies \text{factorial_ A} &= \text{factorial_ B}\\
                                             &= \text{A}
\end{aligned}
<br>
Also ist <code>A</code> Fixpunkt der Funktion <code>factorial_</code> und es gilt

<p style="text-align: center;"><em><code>factorial_(A factorial_) = A factorial_ </code></em></p>
<p style="text-align: left;">und somit per Symmetrie auch</p>
<p style="text-align: center;"><em><code>A factorial_ = factorial_(A factorial_)</code></em>.</p>

<div style="text-align: justify">Wenn wir <code>factorial_</code> zu einer generischen Funktion <code>f</code> abstrahieren, können wir obige Gleichung ja zum Beispiel so implementieren:</div>

In [9]:
def A(f):
    return f(A(f))

<div style="text-align: justify">Als Demonstration, dass dies auch funktioniert (ignoriere fürs Erste die hinzugefügte Lambda-Abstraktion; dazu kommen wir noch):</div>

In [10]:
def A(f):
    return lambda z : f(A(f))(z)

factorial = A(lambda f : lambda n : 1 if not n else n*f(n-1))

factorial(5)

120

<div style="text-align: justify">Hier haben wir jetzt allerdings Zuweisung und Selbstreferenz drin, die wir ja vermeiden wollten. Aber das können wir einfach beheben. Bauen wir mal einen äquivalenten Lambda-Term. Dieser braucht als erstes Argument auf jeden Fall schon mal <em>f</em>:</p>
<p style="text-align: center;">$\lambda f. ??$</div>
<div style="text-align: justify">Ferner muss <em>f</em> irgendwie verbaut werden, sodass am Ende eine Iteration der Form <em>f(f(f(..)))</em> herauskommt. Wir wissen vom ω-Kombinator ja schon, dass</div>
<br>
$$(\lambda x. xx)(\lambda x. xx) \rightarrow (\lambda x. xx)(\lambda x. xx)$$
<br>
<div style="text-align: justify">Na dann können wir doch einfach stattdessen den Term <em>λ. x. f(xx)</em> verwenden, denn</div>
<br>

\begin{aligned}
  (\lambda x. f(xx))(\lambda x. f(xx)) & \rightarrow f((\lambda x. f(xx))(\lambda x. f(xx)))\\
                                       & \rightarrow f(f((\lambda x. f(xx))(\lambda x. f(xx))))\\
                                       & \rightarrow f(f(f(\dots)))
\end{aligned}
<br>

<div style="text-align: justify">Der Term lautet insgesamt also
<br>
<br>
$$\lambda f. (\lambda x. f(xx))(\lambda x. f(xx))$$
<br>
Und dies ist der <strong>Y-Kombinator.</strong> Dieser Term ist jetzt also (fast) äquivalent zur Funktion <code>A</code>; <code>A</code> nimmt eine Funktion und iteriert diese und der Y-Kombinator macht das gleiche.
<br>
Man kann den Y-Kombinator als eine Art Generator betrachten, den man mit einer Funktion <em>f</em> (wie etwa <code>factorial_</code>) programmiert. Sofern <em>f</em>’s erstes Argument im Rumpf von <em>f</em> auftritt, sorgt der Generator von allein dafür, dass er im Rumpf von <em>f</em> wieder auftritt und stellt außerdem zu jedem Zeitpunkt die Definition von <em>f</em> zur Verfügung. Wenn <em>f</em> keine weiteren Argumente hat, passiert nichts außer dieser Schleife. Wenn <em>f</em> aber noch andere Argumente hat (also innere Funktionen -- Currying), können so rekursive Berechnungen mit diesen durchgeführt werden, denn an der Position des ersten Arguments von <em>f</em> im Rumpf von <em>f</em> steht jetzt dank des Y-Kombinators wieder <em>f</em>’s innere Funktion zur Verfügung.</div>
<br>
<div style="text-align: justify">Um den letzten Absatz genau zu verstehen, schauen wir uns jetzt nochmal im Detail an, wie der Y-Kombinator arbeitet.
<br>
Sagen wir, wir haben eine Funktion, die wir metasprachlich (denn wir haben ja keine Zuweisung) φₒ nennen. φₒ hat eine innere Funktion φᵢ, die einen Vektor <em>x̄</em> von Argumenten nimmt:
<br>
<br>
$$\overbrace{\lambda f. \underbrace{\lambda \vec{x}. \text{ if ... then } v \text{ else } h(f(\vec{x}'))}_{\varphi_i}}^{\varphi_o}$$
<br>
<em>λx̄</em>  denotiert hier einfach $\lambda x_1. \lambda x_2. \dots \lambda x_n$, wobei <em>n = |x̄|</em>. <em>x̄'</em> wird weiter untern erklärt und <em>h</em> ist die Funktion, die die eigentliche Kernoperation ausführt, also etwa <em>x ⋅ f(x')</em>.
<br>
Die Definition des Y-Kombinators lautet wie oben:
<br>
<br>
$$\text{Y} \equiv \lambda f . (\lambda x. f(xx)) (\lambda x. f(xx))$$
<br>
Wenn wir <em>Y</em> auf eine Funktion <em>g</em> anwenden, erhalten wir folgende Ableitung:</div>
<br>
\begin{aligned}
  \text{Y} g & \rightarrow (\lambda x. g(xx)) (\lambda x. g(xx)) \\
           & \rightarrow g((\lambda x. g(xx)) (\lambda x. g(xx))) \\
           & \rightarrow g((\lambda f . (\lambda x. f(xx)) (\lambda x. f(xx)))g)\\
           & \equiv g(Yg)
\end{aligned}
<br>
<div style="text-align: justify">Jetzt können wir untenstehendes Diagramm verstehen: Wir beginnen damit <em>Y</em> auf <em>φₒ</em> anzuwenden. Gemäß unserer Ableitung ergibt dies <em>φₒ(Y φₒ)</em>. Das wiederum leitet sich zu <em>φᵢ[f/Y φₒ]</em> ab, sprich in <em>φᵢ</em> ist jetzt <em>f</em> mit <em>Y φₒ</em> substituiert. <em>φᵢ[f/Y φₒ]</em> übergeben wir danach die Argumente <em>x̄</em>. <em>φᵢ[f/Y φₒ]</em> transformiert <em>x̄</em> (etwa <em>n ⟶ n - 1</em> für die Fakultätsfunktion) zu <em>x̄'</em> und übergibt es an <em>Y φₒ</em>, das ja an der Stelle des rekursiven Aufrufs innerhalb von <em>φᵢ[f/Y φₒ]</em> steht.</div>
<br>
$$ \varphi_i[f/\text{Y} \varphi_o]\bar{x} \to  \varphi_i[f/\text{Y} \varphi_o]\bar{x}'$$
<br>
<div style="text-align: justify">
<em>Y φₒ </em>leitet sich wiederum zu <em>φᵢ[f/Y φₒ]</em> ab und wir erhalten also <em>φᵢ[f/φᵢ[f/Y φₒ]x̄']</em>. Diesen Zyklus im Kontrolldiagramm beschreiten wir solange, bis die Abbruchbedingung von <em>φᵢ </em>erreicht ist, denn dann geben wir den Basisfallwert <em>v</em> zurück und ehalten also einen Ausdruck</div>
<br>
$$\varphi_i[f/\varphi_i[f/\dots \varphi_i[f/\text{Y} \varphi_o]\bar{x}' \dots]]$$
<br>
<br>
<figure class="image"><img src="img/y_dia-1200x469.png" alt="{{ include.description }}"><figcaption style="text-align: center"><font size="3">Control-Flow-Diagramm für den Y-Kombinator</font></figcaption></figure>
<br>
<div style="text-align: justify">
Als nächstes können wir den Y-Kombinator ja mal in Python ausprobieren:</div>

In [11]:
Y = lambda f : (lambda x : f(x(x)))(lambda x : f(x(x)))

factorial_ = lambda f : lambda n : 1 if not n else n*f(n-1)

factorial = Y(factorial_)

RecursionError: maximum recursion depth exceeded

<div style="text-align: justify">Das funktioniert wohl so einfach nicht. Grund ist die zuvor erwähnte strenge Auswertungsreihenfolge. Der Interpreter steigt in eine Rekursion ab, die im Diagramm dem Knoten <em>φₒ(Y φₒ)</em>  entspricht:
<br>
<br>
<p style="text-align: center;"><em>φₒ(Y φₒ)</em> ⟶ <em>φₒ(</em><em>φₒ(Y φₒ)) </em>⟶ <em>φₒ(</em><em>φₒ(φₒ(Y φₒ))) ⟶ ...</em></p>
<br>
Wir müssen die Auswertung des Arguments <em>Y φₒ von φₒ</em> also irgendwie aufhalten. Zu diesem Zweck hatte ich in der Funktion <code>A</code> die Lambda-Abstraktion eingebaut. Diese verhindert, dass der rekursive Aufruf vor dem Aufruf der äußeren Funktion ausgeführt wird. Hier nochmal der Code:</div>

In [ ]:
def A(f):
    return lambda z : f(A(f))(z)

factorial = A(lambda f : lambda n : 1 if not n else n*f(n-1))

factorial(5)

<div style="text-align: justify">Das äußere Lambda erwartet also ein Argument, das an den Parameter <code>z</code> überwiesen wird, und reicht es weiter an den rekursiven Aufruf. Dieses Argument ist <code>n - 1</code>. Also findet die Rekursion erst statt, wenn wir sie auch wirklich brauchen &nbsp;&ndash; beim rekursiven Aufruf der Fakultätsfunktion.
Wenn wir nun einen Lambda-Term konstruieren wollen, der ebenso dieses verzögerte Rekursionsverhalten aufweist, müssen wir also wohl den Y-Kombinator an mindestens einer Stelle auch um eine zusätzliche Lambda-Abstraktion ergänzen. Der Kern des Y-Kombinators ist ja der Term <em>λ x. f(xx).</em> Und wir haben auch festgehalten, dass
<br>
<br>
<p style="text-align: center;"><em>(λ</em><em> x. f(xx))(λ x. f(xx)) ⟶ f((λ x. f(xx))(λ x. f(xx))) ⟶ ...</em></p>
<br>
Insofern muss die Lambda-Abstraktion ja den Term <em>(λ</em><em> x. f(xx))(λ x. f(xx))</em> umschließen, damit dieser nicht sofort zu <em>f((λ</em><em> x. f(xx))(λ x. f(xx)))</em> evaluiert. Wir könnten jetzt direkt um den ersten <em>(λ</em><em> x. f(xx))(λ x. f(xx))</em> Term, der in der Definition des Y-Kombinators steht, abstrahieren. Das bringt uns aber nicht viel, denn nach applikativer Auswertungsreihenfolge erhalten wir
</div>
<br>
\begin{aligned}
                                                \text{Y}' & \equiv \lambda f . \lambda z. (\lambda x. f(xx)) (\lambda x. f(xx)) z \\
\\
                                              \text{Y}' g & \rightarrow \lambda z. (\lambda x. g(xx)) (\lambda x. g(xx)) z\\
\\
  (\lambda z. (\lambda x. g(xx))(\lambda x. g(xx)) z) v &\rightarrow (\lambda x. g(xx))(\lambda x. g(xx)) v\\
                                                        &\rightarrow g((\lambda x. g(xx))(\lambda x. g(xx))) v\\
                                                        &\rightarrow g(g((\lambda x. g(xx))(\lambda x. g(xx)))) v\\
                                                        &\rightarrow \dots
\end{aligned}
<br>

<div style="text-align: justify">und haben somit nichts gewonnen, außer die Endlosrekursion einen Schritt lang hinauszuzögern. Wenn wir genau hinsehen, erkennen wir aber jetzt schon, wo wir die Abstraktion eigentlich einfügen müssen, um den gewünschten Effekt zu erzielen: Nämlich in den rechten <em>λ x. f(xx)</em>-Term; und zwar um den <em>xx</em>-Teilterm, denn der rechte <em>λ x. f(xx)</em>-Term wird ja vom linken <em>λ x. f(xx)</em>-Term dupliziert. Den <em>xx</em>-Teilterm umschließen muss die Abstraktion, da dieser ja in ein paar Schritten wieder zum linken "Duplikator-Term" wird, der das Resultat des Dupliziervorgangs ja eben in die Abstraktion mit <em>z</em> einbinden soll. Und damit wir gleich beim ersten Schritt die Evaluationsverzögerung haben und alles schön symmetrisch ist, bauen wir die Abstraktion auch noch in den linken Term ein:
<br>
<br>
<p style="text-align: center;"><em>λ f. (λ x. f(λ z. xxz))(λ x. f(λ z. xxz))</em></p>
<br>
Dieser Term wird <strong>Z-Kombinator</strong> genannt und ist der Y-Kombinator für formale Systeme, die die applikative Evaluationsreihenfolge verwenden. Zur Demonstration, dass sowohl die obige Variante als auch die Variante <em>λ f. λ z. (λ x. f(xx))(λ x. f(λ z. xxz)) z</em> funktionieren:</div>

In [ ]:
Z = lambda f : (lambda x : f(lambda z : x(x)(z))) (lambda x : f(lambda z : x(x)(z)))

factorial_ = lambda f : lambda n : 1 if not n else n*f(n-1)

factorial_Z = Z(factorial_)

factorial_Z(5)

In [ ]:
Z_ = lambda f : lambda z : (lambda x : f(x(x)))(lambda x : f(lambda z : x(x)(z)))(z)

factorial_ = lambda f : lambda n : 1 if not n else n*f(n-1)

factorial_Z_ = Z_(factorial_)

factorial_Z_(5)

<div style="text-align: justify">Zuletzt schauen wir uns nochmal eine formale Ableitung der Programmierung des Z-Kombinators mit einer generischen Funktion <em>g</em> sowie ein Control-Flow-Diagramm für diesen an, das fast identisch zum obigen Diagramm ist und die gleiche Funktion <em>φₒ</em> benutzt.</div>

\begin{aligned}
       \text{Z} & \equiv \lambda f . (\lambda x. f( \lambda z . xxz)) (\lambda x. f( \lambda z. xxz)) \\
\\
     \text{Z} g & \rightarrow (\lambda x. g( \lambda z . xxz)) (\lambda x. g( \lambda z. xxz)) \\
              & \rightarrow g( \lambda z . (\lambda x. g( \lambda z. xxz))(\lambda x. g( \lambda z. xxz))z) \\
              & \rightarrow g( \lambda z . (\lambda f . (\lambda x. f( \lambda z . xxz)) (\lambda x. f( \lambda z. xxz))) g z) \\
              & \equiv g(\lambda z. \text{Z} g z)
\end{aligned}

<figure class="image"><img src="img/z_dia-1200x469.png" alt="{{ include.description }}"><figcaption style="text-align: center"><font size="3">Control-Flow-Diagramm für den Z-Kombinator</font></figcaption></figure>
<br>
<div style="text-align: justify">Wir sehen im Knoten <em>φₒ(λ z. Z φₒ z)</em> der linken Tafel den entscheidenden Unterschied zum Diagramm des Y-Kombinators -- dieser Term weist die Lambda-Abstraktion auf, die die verfrühte Rekursion verhindert. In der rechten Tafel ist zu sehen, wie das Argument <em>x̄'</em> an den Parameter <em>z</em> übergeben wird, was die Verzögerungs-Abstraktion aufhebt, wodurch nach applikativer Reihenfolge der Ausdruck <em>Z φₒ</em> evaluiert wird, was als Top-Level-Ausdruck wiederum <em>φᵢ</em> erzeugt und wir erhalten <em>φᵢ x̄'</em>, welches dann seinerseits entweder den Basisfallwert liefert, oder in die Rekursion geht. Im rekursiven Aufruf steht dann natürlich wieder der Term <em>λ z. Z φₒ z</em>, denn dieser wird ja im Ausdruck <em>φₒ(λ z. Z φₒ z)</em> in <em>φᵢ </em>hineinkopiert.</div>
<br>
<br>

<h4>Zusammenfassung</h4>
<hr />
<div style="text-align: justify">ir haben begonnen damit, das Problem der Rekursion in anonymen Kalkülen zu benennen. Wir haben schnell einen einfachen Mechanismus gefunden, dieses zu lösen; den Fixpunkt einer Funktion zu finden und sie darauf zu iterieren. Dazu mussten wir der Funktion ihre eigene Definition übergeben und sie so aufbauen, dass sie diese an sich selbst weitergibt. Danach haben wir festgestellt, dass wir diese Mechanik teilweise auslagern können, und zwar mittels des ω-Kombinators. Dieser verlangte aber, dass wir unsere Funktionen auf eine etwas unnatürliche Weise definieren, um die Weitergabe der eigenen Definitionen zu ermöglichen. Deswegen haben wir einen Kombinator entwickelt, der uns dies abnimmt -- den Y-Kombinator. Daraufhin haben wir aber feststellen müssen, dass dieser ein Problem in Kalkülen mit applikativer Evaluationsstrategie darstellt. Für dieses Problem konnten wir jedoch eine einfache Lösung finden, nämlich den problematischen Teilausdruck in eine Lambda-Abstraktion einzubetten, was die verfrühte Rekursion aufhielt. Dieser abgewandelte Y-Kombinator war der Z-Kombinator.
Mit diesen beiden Kombinatoren haben wir jetzt Werkzeuge, um elegant rekursive Funktionen schreiben zu können, ohne auf Zuweisung als Bestandteil des Kalküls, in dem wir uns befinden, angewiesen zu sein..</div>